Import libraries

In [ ]:
import pathlib
from tensorflow import keras
from tensorflow.keras import layers


Create a CNN model

In [ ]:
# The depth of the feature maps progressively increases in the model (from 32
# to 256), whereas the size of the feature maps decreases (from 180 × 180 to
# 7 × 7). This is a pattern you’ll see in almost all convnets.

# We choose 180 by 180 for the input choice (a somewhat arbitrary choice)
inputs = keras.Input(shape=(180, 180, 3))

# Start the model with a Rescaling layer, which will rescale
# image inputs (whose values are originally in the [0, 255] range) to the [0, 1] range.
x = layers.experimental.preprocessing.Rescaling(1./255)(inputs)

x = layers.Conv2D(filters=32, kernel_size=3, activation="relu")(x)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=64, kernel_size=3, activation="relu")(x)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=128, kernel_size=3, activation="relu")(x)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=256, kernel_size=3, activation="relu")(x)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=256, kernel_size=3, activation="relu")(x)
x = layers.Flatten()(x)

# We are looking at a binary-classification problem, we’ll end the model with a single unit
# (a Dense layer of size 1) and a sigmoid activation. This unit will encode the probability that the
# model is looking at one class or the other.
outputs = layers.Dense(1, activation="sigmoid")(x)

model = keras.Model(inputs=inputs, outputs=outputs)

print(model.summary())

Configure the model for training

In [ ]:
# Because we ended the model with a single sigmoid unit, we use binary crossentropy as the loss function
model.compile(loss="binary_crossentropy",optimizer="rmsprop", metrics=["accuracy"])

**Data preprocessing**

- Currently, the data stored in a drive as JPEG files, so the steps for getting it into the model are as follows:
    1. Read the picture files.
    2. Decode the JPEG content to RGB grids of pixels.
    3. Convert these into floating-point tensors.
    4. Resize them to a shared size (we’ll use 180 x 180).
    5. Pack them into batches (we’ll use batches of 32 images).

Keras has utility function  *image_dataset_from_directory* to take care of these steps automatically. It lets you quickly set up a data pipeline that can automatically turn image files on disk into batches of preprocessed tensors.

Calling image_dataset_from_directory(directory) will first list the subdirectories of directory and assume each one contains images from one of your class. It will then index the image files in each subdirectory. Finally, it will create and return a tf.data.Dataset object configured to read these files, shuffle them, decode them to tensors, resize them to a shared size, and pack them into batches.

In [ ]:
from tensorflow.keras.preprocessing import image_dataset_from_directory

new_base_dir = pathlib.Path("../data/cats_vs_dogs_small")

train_dataset = image_dataset_from_directory(new_base_dir / "train", image_size=(180, 180), batch_size=32)
validation_dataset = image_dataset_from_directory( new_base_dir / "validation", image_size=(180, 180), batch_size=32)
test_dataset = image_dataset_from_directory( new_base_dir / "test", image_size=(180, 180), batch_size=32)

**Model Training**
- We fit the model using the training dataset 
- *validation_data* argument in fit() to monitor validation metrics on a separate Dataset object.

- We use a ModelCheckpoint callback to save the model after each epoch. 
    - *filepath=* where to save the file 
    - save_best_only=True and monitor="val_loss" : they tell the callback to only save a new file (overwriting any previous one) when the current value of the      val_loss metric is lower than at any previous time during training. This guarantees that your saved file will always contain the state of the model corresponding to its best-performing training epoch, in terms of its performance on the validation data.

In [ ]:
callbacks = [
    keras.callbacks.ModelCheckpoint(
        filepath="convnet_from_scratch.keras",
        save_best_only=True,
        monitor="val_loss")
]

history = model.fit(
    train_dataset,
    epochs=30,
    validation_data=validation_dataset,
    callbacks=callbacks)

**Visualization**
- Plot the loss and accuracy of the model over the training and validation data during training

In [ ]:
import matplotlib.pyplot as plt
accuracy = history.history["accuracy"]
val_accuracy = history.history["val_accuracy"]
loss = history.history["loss"]
val_loss = history.history["val_loss"]
epochs = range(1, len(accuracy) + 1)
plt.plot(epochs, accuracy, "bo", label="Training accuracy")
plt.plot(epochs, val_accuracy, "b", label="Validation accuracy")
plt.title("Training and validation accuracy")
plt.legend()
plt.figure()
plt.plot(epochs, loss, "bo", label="Training loss")
plt.plot(epochs, val_loss, "b", label="Validation loss")
plt.title("Training and validation loss")
plt.legend()
plt.show()

**Evaluation**
- Evaluate the model on Test set

In [ ]:
test_model = keras.models.load_model("convnet_from_scratch.keras")
test_loss, test_acc = test_model.evaluate(test_dataset)
print(f"Test accuracy: {test_acc:.3f}")

**Data Augmentation**

Data augmentation takes the approach of generating more training data from existing training samples, by augmenting the samples via a number of random transformations that yield believable-looking images. The goal is that at training time, your model will never see the exact same picture twice. This helps expose the model to more aspects of the data and generalize better.

- RandomFlip("horizontal") will apply horizontal flipping to a random 50% of the images that go through it.
- RandomRotation(0.1) will rotation the input images by a random value in the range [-10%, +10%]
- RandomZoom(0.2) will zoom in or out of the image by a random factor in the range [-20%, +20%].

In Keras, this can be done by adding a number of data augmentation layers at the start of your model.

In [ ]:
data_augmentation = keras.Sequential(
    [
        layers.experimental.preprocessing.RandomFlip("horizontal"),
        layers.experimental.preprocessing.RandomRotation(0.1),
        layers.experimental.preprocessing.RandomZoom(0.2),
    ]
)

Display some randomly augmented training images

In [ ]:
plt.figure(figsize=(10, 10))
for images, _ in train_dataset.take(1):
    for i in range(9):
        augmented_images = data_augmentation(images)
        ax = plt.subplot(3, 3, i + 1)
        plt.imshow(augmented_images[0].numpy().astype("uint8"))
        plt.axis("off")

Defining a new convnet that includes image augmentation and dropout

In [ ]:
inputs = keras.Input(shape=(180, 180, 3))
x = data_augmentation(inputs)
x = layers.experimental.preprocessing.Rescaling(1./255)(x)
x = layers.Conv2D(filters=32, kernel_size=3, activation="relu")(x)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=64, kernel_size=3, activation="relu")(x)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=128, kernel_size=3, activation="relu")(x)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=256, kernel_size=3, activation="relu")(x)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=256, kernel_size=3, activation="relu")(x)
x = layers.Flatten()(x)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(1, activation="sigmoid")(x)
model = keras.Model(inputs=inputs, outputs=outputs)

model.compile(loss="binary_crossentropy",optimizer="rmsprop",metrics=["accuracy"])

In [ ]:
callbacks = [
    keras.callbacks.ModelCheckpoint(
        filepath="convnet_from_scratch_with_augmentation.keras",
        save_best_only=True,
        monitor="val_loss")
]

history = model.fit(
train_dataset,
epochs=100,
validation_data=validation_dataset,
callbacks=callbacks)

Evaluating the model on the test set

In [ ]:
test_model = keras.models.load_model("convnet_from_scratch_with_augmentation.keras")
test_loss, test_acc = test_model.evaluate(test_dataset)
print(f"Test accuracy: {test_acc:.3f}")

In [ ]:
import matplotlib.pyplot as plt
accuracy = history.history["accuracy"]
val_accuracy = history.history["val_accuracy"]
loss = history.history["loss"]
val_loss = history.history["val_loss"]
epochs = range(1, len(accuracy) + 1)
plt.plot(epochs, accuracy, "bo", label="Training accuracy")
plt.plot(epochs, val_accuracy, "b", label="Validation accuracy")
plt.title("Training and validation accuracy")
plt.legend()
plt.figure()
plt.plot(epochs, loss, "bo", label="Training loss")
plt.plot(epochs, val_loss, "b", label="Validation loss")
plt.title("Training and validation loss")
plt.legend()
plt.show()